<a href="https://colab.research.google.com/github/sm2hashmi/Syed-Muhammad-Masab-Hashmi/blob/master/Deep%20Learning%20Project%202.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy 
import pandas
from keras import models
from keras import layers
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.utils import shuffle


In [0]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)


In [3]:
# load dataset
from google.colab import files
uploaded = files.upload()
import io
dataframe = pandas.read_csv(io.BytesIO(uploaded['iris.csv']))
dataset = dataframe.values
X = dataset[:,0:4].astype(float)
Y = dataset[:,4]


Saving iris.csv to iris.csv


In [0]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)


In [0]:
def create_model():
   model = Sequential()
   model.add(layers.Dense(8, activation= 'relu', input_shape = (4,)))
   model.add(layers.Dense(3, activation = 'softmax'))
    
   model.compile(optimizer='Adam', loss ='categorical_crossentropy', metrics = ['accuracy'])
  
   return model

In [0]:
estimator = KerasClassifier(build_fn=create_model, epochs=200, batch_size=5, verbose=0)
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))



Baseline: 97.33% (5.33%)


In [0]:
numpy.random.seed(seed)
estimators = []

estimators.append(('mlp', KerasClassifier(build_fn=create_model, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Standardized: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Standardized: 95.95% (4.45%)


In [0]:
def create_large():
   model = Sequential()
   model.add(layers.Dense(16, activation= 'relu', input_shape = (4,)))
   model.add(layers.Dense(3, activation = 'softmax'))
    
   model.compile(optimizer='Adam', loss ='categorical_crossentropy', metrics = ['accuracy'])
  
   return model
  
estimators = []
estimators.append(('mlp', KerasClassifier(build_fn=create_large, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Larger: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Larger: 96.62% (4.51%)


**Using Functional API**

In [0]:
from keras.layers import Input
from keras.models import Model
from keras import layers

def create_network():
  visible = Input(shape = (4,))
  hidden1 = layers.Dense(8, activation = 'relu')(visible)
  output = layers.Dense(3, activation= 'softmax')(hidden1)
  model = Model(inputs=visible, outputs=output)
  model.compile(optimizer='Adam', loss ='categorical_crossentropy',metrics = ['accuracy'])
  return model

In [24]:
numpy.random.seed(seed)
estimators = []
estimators.append(('mlp', KerasClassifier(build_fn=create_network, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("STandard: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

STandard: 95.95% (4.45%)


In [0]:
def create_larger():
  visible = Input(shape = (4,))
  hidden1 = layers.Dense(16, activation = 'relu')(visible)
  output = layers.Dense(3, activation= 'softmax')(hidden1)
  model = Model(inputs=visible, outputs=output)
  model.compile(optimizer='Adam', loss ='categorical_crossentropy',metrics = ['accuracy'])
  return model

In [26]:
numpy.random.seed(seed)
estimators = []
estimators.append(('mlp', KerasClassifier(build_fn=create_network, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Larger: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Larger: 96.62% (4.51%)


In [0]:
def create_smaller():
  visible = Input(shape = (4,))
  hidden1 = layers.Dense(3, activation = 'relu')(visible)
  output = layers.Dense(3, activation= 'softmax')(hidden1)
  model = Model(inputs=visible, outputs=output)
  model.compile(optimizer='Adam', loss ='categorical_crossentropy',metrics = ['accuracy'])
  return model

In [30]:
numpy.random.seed(seed)
estimators = []
estimators.append(('mlp', KerasClassifier(build_fn=create_network, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Smaller: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Smaller: 96.62% (4.51%)


**Using Model Sub-Classing API**

In [32]:
import keras
import tensorflow.contrib.eager as tfe

class MyBaselineModel(keras.Model):
  
  def __init__(self):
    super(MyBaselineModel,self).__init__('my_baseline')
  #  self.units = 60
    self.dense1 = keras.layers.Dense(8, activation='relu')
    self.dense2 = keras.layers.Dense(3, activation='softmax')
    
  def call(self, inputs):
    x = self.dense1(inputs)
    return self.dense2(x)
    
def create_baseline_subclass_model():
  inputs = keras.Input(shape=(4,))
  mymodel = MyBaselineModel()
  outputs = mymodel.call(inputs)
    
  model = keras.Model(inputs,outputs)
    
# compiling model
  model.compile(optimizer = 'Adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

  return model

# evaluate baseline model with standardized dataset
numpy.random.seed(seed)
estimators = []
estimators.append(('mlp', KerasClassifier(build_fn=create_baseline_subclass_model, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Standardized: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Standardized: 95.95% (4.45%)


In [34]:
class MySmallerModel(keras.Model):
  
  def __init__(self):
    super(MySmallerModel,self).__init__('my_Smaller')
    self.dense1 = keras.layers.Dense(4, activation='relu')
    self.dense2 = keras.layers.Dense(3, activation='softmax')
    
  def call(self, inputs):
    x = self.dense1(inputs)
    return self.dense2(x)
    
def create_smaller_subclass_model():
  inputs = keras.Input(shape=(4,))
  mymodel = MySmallerModel()
  outputs = mymodel.call(inputs)
    
  model = keras.Model(inputs,outputs)
    
# compiling model
  model.compile(optimizer = 'Adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

  return model

# evaluate smaller model with standardized dataset
estimators = []
estimators.append(('mlp', KerasClassifier(build_fn=create_smaller_subclass_model, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Smaller: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Smaller: 85.86% (14.81%)


In [36]:
class MylargerModel(keras.Model):
  
  def __init__(self):
    super(MylargerModel,self).__init__('my_larger')
    self.dense1 = keras.layers.Dense(8, activation='relu')
    self.dense2 = keras.layers.Dense(8, activation='relu')
    self.dense3 = keras.layers.Dense(3, activation='softmax')
    
  def call(self, inputs):
    x = self.dense1(inputs)
    x = self.dense2(x)
    return self.dense3(x)
    
def create_larger_subclass_model():
  inputs = keras.Input(shape=(4,))
  mymodel = MylargerModel()
  outputs = mymodel.call(inputs)
    
  model = keras.Model(inputs,outputs)
    
# compiling model
  model.compile(optimizer = 'Adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

  return model

# evaluate larger model with standardized dataset
estimators = []
estimators.append(('mlp', KerasClassifier(build_fn=create_larger_subclass_model, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Larger: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Larger: 98.00% (3.06%)


**Builing K-Fold Validation without Scikit**

In [0]:
def create_model():
   model = models.Sequential()
   model.add(layers.Dense(8, activation= 'relu', input_shape = (X.shape[1],)))
   model.add(layers.Dense(4, activation ='relu'))  
   model.add(layers.Dense(3, activation = 'softmax'))
    
   model.compile(optimizer='Adam', loss ='categorical_crossentropy', metrics = ['accuracy'])
  
   return model

In [0]:
# shuffling data
dataframe = shuffle(dataframe)

# checking data
dataset = dataframe.values

# split into input (X) and output (Y) variables
X = dataset[:,0:4].astype(float)
Y = dataset[:,4]

# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
reverse_encoding = encoder.inverse_transform(encoded_Y) # reversing the encoding


In [61]:
k = 10
num_val_samples = len(X) // 10
num_epochs = 100
all_scores = numpy.array([])

for i in range(k):
  print('processing fold # ', i+1)
  # prepare the validation data: data from partition # k
  val_data = X[i * num_val_samples: (i + 1) * num_val_samples]
  val_targets = encoded_Y[i * num_val_samples: (i + 1) * num_val_samples]
  
  # prepare the training data: data from data - k
  partial_train_data = numpy.concatenate(                    
      [X[:i * num_val_samples],
      X[(i + 1 ) * num_val_samples:]],
  axis = 0)
  partial_train_targets = numpy.concatenate(
      [encoded_Y[:i * num_val_samples],
      encoded_Y[(i + 1 ) * num_val_samples:]],
  axis = 0)
  
  # Build the Keras Models (already commpiled)
  model = create_model()
  
  # Train the model (in silence mode, verbose = 0)
  history = model.fit(partial_train_data, partial_train_targets, epochs = num_epochs, batch_size = 5, verbose = 0)
  
  # Evaluate the model on the validation data
  val_categorical_crossentropy, val_adam = model.evaluate(val_data, val_targets, verbose = 0)
  
  all_scores = numpy.append(all_scores, val_adam)
  print("K-Fold Validation : (Accuracy)Mean %.2f%% (Error)Std (%.2f%%)" % (all_scores.mean()*100, all_scores.std()*100))

processing fold #  3


ValueError: ignored